# Deploy DeepSeek  R1 Models on Amazon Bedrock using Custom Model Imports

## DeepSeek-R1-Distill models are fine-tuned based on open-source models, using samples generated by DeepSeek-R1. 

Out of 6 models, following two are based on Llama architecture. 
- DeepSeek-R1-Distill-Llama-8B 
- Llama-3.3-70B-Instruct 

In this notebook we will deploy models based on Llama architecture

### Prerequisites 
* AWS CLI installed: [Installing or updating to the latest version of the AWS CLI](https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html)

### Install required Dependencies

In [ ]:
%pip install huggingface_hub[hf_transfer] ipywidgets boto3
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
# faster downloads
os.environ["HF_HUB_ENABLE_HF_TRANSFER"]="1"

In [ ]:
import os
from huggingface_hub import snapshot_download

# Get current working directory
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

import ipywidgets as widgets
from IPython.display import display
import os

# Define available models
available_models = {
    "DeepSeek-R1-Distill-Llama-8B": "deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
    "DeepSeek-R1-Distill-Llama-70B": "deepseek-ai/DeepSeek-R1-Distill-Llama-70B",
}

# Create global variables for selected model
global selected_model_name
global selected_model_id
selected_model_name = "DeepSeek-R1-Distill-Llama-8B"
selected_model_id = available_models[selected_model_name]

# Create model dropdown
model_dropdown = widgets.Dropdown(
    options=list(available_models.keys()),
    value=selected_model_name,
    description='Model:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create select button
select_button = widgets.Button(
    description='Select Model',
    style={'description_width': 'initial'},
    button_style='success',
    layout={'width': 'auto'}
)

# Create output widget for status messages
output = widgets.Output()

def on_select_button_click(b):
    global selected_model_name
    global selected_model_id
    with output:
        output.clear_output()
        selected_model_name = model_dropdown.value
        selected_model_id = available_models[selected_model_name]
        print(f"Selected model: {selected_model_name}")
        print(f"Model ID: {selected_model_id}")

select_button.on_click(on_select_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>Select Model</h3>"),
    model_dropdown,
    select_button,
    output
]))


In [16]:
model_path = os.path.join(current_dir, selected_model_name)

In [ ]:
# Model ID picked up from the cell above, 
model_id = selected_model_id


try:
    # Download the model
    local_dir = snapshot_download(
        repo_id=model_id,
        local_dir=model_path,
        ignore_patterns=["*.md", "*.h5"],  
        local_dir_use_symlinks=False 
    )
    print(f"Model downloaded successfully to: {local_dir}")
except Exception as e:
    print(f"An error occurred during download: {e}")



In [ ]:
import boto3
import ipywidgets as widgets
from IPython.display import display
import os

# Set default region
default_region = 'us-east-1'
boto3.setup_default_session(region_name=default_region)

# Create a global variable to store the selected region
global selected_region
selected_region = default_region

def get_available_regions():
    """Get list of available AWS regions"""
    ec2_client = boto3.client('ec2', region_name=default_region)
    regions = [region['RegionName'] for region in ec2_client.describe_regions()['Regions']]
    return sorted(regions)

# Create region dropdown
region_dropdown = widgets.Dropdown(
    options=get_available_regions(),
    value=default_region,
    description='Region:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create select button
select_button = widgets.Button(
    description='Select Region',
    style={'description_width': 'initial'},
    button_style='success',
    layout={'width': 'auto'}
)

# Create output widget for status messages
output = widgets.Output()

def on_select_button_click(b):
    global selected_region
    with output:
        output.clear_output()
        selected_region = region_dropdown.value
        print(f"Selected region: {selected_region}")
        # Update boto3 session with new region
        boto3.setup_default_session(region_name=selected_region)

select_button.on_click(on_select_button_click)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>Select AWS Region</h3>"),
    region_dropdown,
    select_button,
    output
]))


In [ ]:
def get_buckets_in_region(region):
    """Get list of S3 buckets in the specified region"""
    s3_client = boto3.client('s3', region_name=region)
    buckets = []
    try:
        response = s3_client.list_buckets()
        for bucket in response['Buckets']:
            try:
                bucket_region = selected_region
                # Note: us-east-1 returns None instead of 'us-east-1'
                if bucket_region is None:
                    bucket_region = 'us-east-1'
                if bucket_region == region:
                    buckets.append(bucket['Name'])
            except Exception as e:
                print(f"Error getting location for bucket {bucket['Name']}: {e}")
                continue
    except Exception as e:
        print(f"Error listing buckets: {e}")
    return sorted(buckets)


# Create bucket dropdown
bucket_dropdown = widgets.Dropdown(
    options=get_buckets_in_region(selected_region),
    description='S3 Bucket:',
    style={'description_width': 'initial'},
    layout={'width': 'auto'}
)

# Create refresh button for buckets
refresh_button = widgets.Button(
    description='Refresh Buckets',
    style={'description_width': 'initial'},
    button_style='info'
)

def on_refresh_click(b):
    bucket_dropdown.options = get_buckets_in_region(selected_region)
    bucket_dropdown.value = None

refresh_button.on_click(on_refresh_click)


# Display bucket selection and upload controls
display(widgets.VBox([
    widgets.HTML("<h3>Select S3 Bucket and Upload</h3>"),
    bucket_dropdown,
    widgets.HBox([refresh_button])
]))


In [35]:
s3_bucket_name = bucket_dropdown.value
model_s3_location = os.path.join(s3_bucket_name,selected_model_name)
s3_model_uri = 's3://{}/'.format(model_s3_location)

In [ ]:
# Upload model to Amazon S3 bucket
!aws s3 cp --recursive {model_path} {s3_model_uri} --exclude ".cache/*"

### Create IAM role

In [31]:
session = boto3.Session()
sts_client = session.client('sts')
account_id = sts_client.get_caller_identity()['Account']

In [32]:
bedrock_client = boto3.client(service_name="bedrock")

In [33]:
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "bedrockimport",
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole",
            "Condition": {
                "StringEquals": {
                    "aws:SourceAccount": account_id
                },
                "ArnEquals": {
                    "aws:SourceArn": f"arn:aws:bedrock:{selected_region}:{account_id}:model-import-job/*"
                }
            }
        }
    ]
}


In [ ]:
s3_access_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "s3Access",
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                f"arn:aws:s3:::{s3_bucket_name}",
                f"arn:aws:s3:::{s3_bucket_name}/*"
            ],
            "Condition": {
                "StringEquals": {
                    "aws:ResourceAccount": account_id
                }
            }
        }
    ]
}


In [37]:
import boto3
import json
from typing import Dict, Any

def create_iam_role(role_name: str, description: str, trust_policy: Dict):
    """
    Creates an IAM role that can be assumed by the specified AWS service
    
    Args:
        role_name (str): Name of the IAM role to create
        description (str): Description for the IAM role
        trust_policy (dict): Trust policy that can assume this role
    
    Returns:
        str: ARN of the created role
    """
    # Create IAM client
    iam = boto3.client('iam')
    
   
    
    try:
        # Create the IAM role
        response = iam.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description=description
        )
        
        role_arn = response['Role']['Arn']
        print(f"Successfully created role {role_name} with ARN: {role_arn}")
        
        return role_arn
        
    except iam.exceptions.EntityAlreadyExistsException:
        print(f"Role {role_name} already exists")
        return iam.get_role(RoleName=role_name)['Role']['Arn']
    except Exception as e:
        print(f"Error creating role: {str(e)}")
        raise

def create_iam_policy(policy_name: str, description: str, policy_document: Dict) -> Dict[str, Any]:
    """
    Creates an IAM policy based on the provided policy description
    
    Args:
        policy_name (str): Name of the policy
        description (str): Description of the policy
        policy_document (dict): Dictionary containing the policy document with Version and Statement
    
    Returns:
        Dict containing the created policy details including ARN
    """
    try:
        # Create IAM client
        iam = boto3.client('iam')
        
        # Create the policy
        response = iam.create_policy(
            PolicyName=policy_name,
            Description=description,
            PolicyDocument=json.dumps(policy_document)
        )
        
        print(f"Successfully created policy {policy_name}")
        return response['Policy']
        
    except Exception as e:
        print(f"Error creating policy: {str(e)}")
        raise


def attach_policy_to_role(role_name, policy_arn):
    """
    Attaches an IAM policy to the specified role
    
    Args:
        role_name (str): Name of the IAM role
        policy_arn (str): ARN of the policy to attach
    """
    iam = boto3.client('iam')
    
    try:
        iam.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
        print(f"Successfully attached policy {policy_arn} to role {role_name}")
    except Exception as e:
        print(f"Error attaching policy: {str(e)}")
        raise


In [ ]:
role_name = "BedrockModelImportRole"
role_arn = create_iam_role(role_name, "Role to import model to Amazon Bedrock", trust_policy)
iam_policy = create_iam_policy("BedrockImportModelS3Access", "Provides Amazon Bedrock read access to Amazon S3 bucket containing model.", s3_access_policy)
attach_policy_to_role(role_name,iam_policy["Arn"])

### Import Model to Amazon Bedrock

In [40]:
response = bedrock_client.create_model_import_job(
    jobName=f'model-import-{selected_model_name}',
    roleArn=role_arn,
    importedModelName=selected_model_name,
    modelDataSource={
        's3DataSource': {
            's3Uri': s3_model_uri
        }
    }
)
jobArn = response["jobArn"]


In [ ]:
# Wait for model import to complete
import time

in_progress = True
while in_progress:
  time.sleep(60)
  response = bedrock_client.get_model_import_job(
    jobIdentifier=jobArn
  )
  status = response["status"]
  in_progress = status == "InProgress"

print(f"Model import job status: {status}")
  

In [42]:
MODEL_ID = response["importedModelArn"]

### Run inference

In [ ]:
import json
import boto3
from botocore.config import Config


REGION_NAME = selected_region

config = Config(
    retries={
        'total_max_attempts': 10, 
        'mode': 'standard'
    }
)
message = "What is the color of the sky?"


session = boto3.session.Session()
br_runtime = session.client(service_name = 'bedrock-runtime', 
                                 region_name=REGION_NAME, 
                                 config=config)
    
try:
    invoke_response = br_runtime.invoke_model(modelId=MODEL_ID, 
                                            body=json.dumps({'prompt': message}), 
                                            accept="application/json", 
                                            contentType="application/json")
    invoke_response["body"] = json.loads(invoke_response["body"].read().decode("utf-8"))
    print(json.dumps(invoke_response, indent=4))
except Exception as e:
    print(e)
    print(e.__repr__())
        
    